In [1]:
import pandas as pd
import numpy as np
import textract
tpath = '/Users/alex/Documents/alexdatasci/data_files/MIDUS_1/' \
            + 'stata/ICPSR_02760/DS0001/02760-0001-' \
            + 'Documentation-scales.pdf'


In [2]:
vartext = textract.process(tpath)
vartext = vartext.splitlines()
vartext = pd.Series(vartext)
vartext = vartext.str.decode('utf-8').str.strip()
vartext.replace('',np.nan,inplace=True)
vartext.dropna(inplace=True)
vartext.reset_index(drop=True,inplace=True)


scaleprefix = pd.Series([1 if ']:' in i 
                         or (str(i).endswith(']') 
                             and '-' not in str(i) 
                             and not str(i).startswith(':')
                            )
                         else 0 for i in vartext])
scalename_alt = pd.Series([1 if str(i).split('[')[0] == ''
                           and ']: (cont' in i else 0 
                           for i in vartext]).iloc[1:]
scalename_alt = scalename_alt.append(pd.Series([0]),ignore_index=True)
scaleheading = pd.Series([1 if str(i).split(' (')[0].isupper()
                          and '[' not in i 
                          and i != 'AND' 
                          and '(' not in str(i).split(' (')[0]
                          and len(i) < 50
                          else 0 for i in vartext])
scalecontents = scaleprefix+scalename_alt+scaleheading
scaleinfo = vartext.loc[scalecontents>0].reset_index(drop=True)
scaleinfo = scaleinfo.loc[9:].reset_index(drop=True)
scaleinfo = pd.Series([i.split(':')[0] for i in scaleinfo ])
scaleinfo = scaleinfo.str.split('[')
scaleinfo = pd.Series([i for j in scaleinfo for i in j])
scaleinfo = scaleinfo[scaleinfo!='']
scaleinfo.reset_index(drop=True,inplace=True)

scalenames = pd.Series([i if str(i).islower()==False 
                        and str(i).isupper()==False
                        else ''
                        for i in scaleinfo])
scaleprefix1 = pd.Series(['Scale Prefix' if str(i).isupper() 
                          and ']' in i
                          else ''
                          for i in scaleinfo])
scaleheading = pd.Series(['Scale Heading' if str(i).split(' (')[0].isupper()
                          and ']' not in i 
                          and i != 'AND' 
                          and '(' not in str(i).split(' (')[0]
                          and len(i) < 50
                          else '' for i in scaleinfo])

scaleinfo = pd.Series([i.split(']')[0] for i in scaleinfo])
scaleinfo = scaleinfo.str.strip()



cat = np.array([None]).repeat(len(scaleinfo))
cat[0] = scaleinfo[0]
for i in list(range(1,len(scaleinfo))):
    if scaleinfo[i].isupper() and ']' not in scaleinfo[i]:
        cat[i]=scaleinfo[i]
    else:
        cat[i]=cat[i-1]

cat = pd.Series(cat)
tvar = scalenames+scaleprefix1+scaleheading

scale = np.array([None]).repeat(len(scaleinfo))
scale[0] = None
for i in list(range(1,len(scaleinfo))):
    if scalenames[i]!='' and scaleheading[i]=='':
        scale[i]=scalenames[i]
    elif scalenames[i]=='' and scaleheading[i]!='':
        scale[i]=None
    else:
        scale[i]=scale[i-1]
scale = pd.Series(scale)

scaledf = pd.DataFrame({
    'id':cat,
    'scale':scale,
    'info':scaleinfo
})
scaledf.head()

# Remove brackets
# scaledf['info']=[i.split(']')[0] if ']' in i and i.isupper() else None for i in scaledf['info']]

# Remove None, keep id if it doesn't have any other info
tlf = scaledf['id'].apply(lambda x: scaledf['id'].value_counts()[x])>1
scaledf = scaledf[~(scaledf['info'].isna() & tlf)]
# Reset Index
scaledf.reset_index(drop=True,inplace=True)

In [3]:
# Extract text 
vartext = textract.process(tpath)
vartext = vartext.splitlines()
vartext = pd.Series(vartext)
vartext = vartext.str.decode('utf-8').str.strip()
vartext.replace('',np.nan,inplace=True)
vartext.dropna(inplace=True)
vartext.reset_index(drop=True,inplace=True)

In [4]:
# Drop preamble
vartext = vartext[177:].reset_index(drop=True)

In [5]:
# Filter, keep only Headings/Scales/Vars
scaleprefix = pd.Series([1 if ']:' in i 
                         or (str(i).endswith(']') 
                             and '-' not in str(i) 
                             and not str(i).startswith(':')
                            )
                         else 0 for i in vartext])
scalename_alt = pd.Series([1 if str(i).split('[')[0] == ''
                           and ']: (cont' in i else 0 
                           for i in vartext]).iloc[1:]
scalename_alt = scalename_alt.append(pd.Series([0]),ignore_index=True)
scaleheading = pd.Series([1 if str(i).split(' (')[0].isupper()
                          and '[' not in i 
                          and i != 'AND' 
                          and '(' not in str(i).split(' (')[0]
                          and len(i) < 50
                          else 0 for i in vartext])
scalecontents = scaleprefix+scalename_alt+scaleheading
vartext = vartext[scalecontents>0].reset_index(drop=True)
vartext.iloc[list(range(0,5))+list(range(30,35))]

0                                            DEPRESSION
1                                      Depressed Affect
2     [A1PDEPAF]: (continuous variable based on 7 it...
3     [A1PDEPAD]: (dummy variable based on QA58, QA5...
4                                             Anhedonia
30                          Paternal Affection [A1SEFA]
31                         Paternal Discipline [A1SEFD]
32              Paternal Model of Generosity [A1SEFMOD]
33                        Parental Affection [A1SEMAPA]
34                           CONFLICT TACTICS INVENTORY
dtype: object

In [6]:
# Some scale names are on a separate line, others are
# listed before the varname.
# First, drop text after "[VAR]:"
vartext = pd.Series([i[0] for i in vartext.str.split(':')])

In [7]:
# Then split examples with "Scale_name [VARNAME]" and flatten
vartext = pd.Series([i for j in vartext.str.split('[') for i in j])
# Drop ''
vartext = vartext[vartext!=''].reset_index(drop=True)

In [8]:
# Make series for heading, scale, and varnames
heading = np.array([None]).repeat(len(vartext))
heading[0] = vartext[0]
for i in range(1,len(vartext)):
    if vartext[i].split('(')[0].isupper() \
        and ']' not in vartext[i]:
        heading[i]=vartext[i]
    else:
        heading[i]=heading[i-1]

scale = np.array([None]).repeat(len(vartext))
scale[0] = None
for i in range(1,len(vartext)):
    if vartext[i].split('(')[0].isupper() \
        and ']' not in vartext[i]:
        scale[i]=None
    elif vartext[i].isupper()==False \
        and vartext[i].islower()==False:
        scale[i]=vartext[i]
    elif vartext[i].isupper() \
        and ']' in vartext[i]:
        scale[i]=scale[i-1]

varnames = [i.split(']')[0] if i.isupper() and ']' in i else None \
    for i in vartext]

# Make df
scaledf = pd.DataFrame({
    'Heading':heading,
    'Scale':scale,
    'Varname':varnames
})

In [9]:
# Clean up dataframe:

# Some headings have no associated scale
# These can be identified by 'None' appearing consecutively under scale
scaledf[28:33]

,Heading,Scale,Varname
28,POSITIVE AND NEGATIVE AFFECT,Negative Affect,A1SNEGAF
29,ALCOHOL SCREENING TEST,None,None
30,PARENTAL AFFECTION IN CHILDHOOD,None,None
31,PARENTAL AFFECTION IN CHILDHOOD,Maternal Affection,None
32,PARENTAL AFFECTION IN CHILDHOOD,Maternal Affection,A1SEMA


In [10]:
# Create temporary filter to ID valid headings that have no associated scale
tempfilt = np.array([False]).repeat(len(vartext))
for i in range(1,len(vartext)):
    if scaledf['Scale'][i]==None \
        and scaledf['Scale'][i-1]==None:
        tempfilt[i-1]=True

In [11]:
# Some scales are not real scales
# They appear when Scale!=None and Varname==None
scaledf[222:225]

,Heading,Scale,Varname
222,BODY INDICES,Body Mass Index,A1SBMI
223,BODY INDICES,"Scales.rtf, p. 97]",None
224,SEEING DOCTORS & MENTAL HEALTH PROFESSIONALS,None,None


In [12]:
# First instances of scale headings are redundant
scaledf.head()

,Heading,Scale,Varname
0,DEPRESSION,None,None
1,DEPRESSION,Depressed Affect,None
2,DEPRESSION,Depressed Affect,A1PDEPAF
3,DEPRESSION,Depressed Affect,A1PDEPAD
4,DEPRESSION,Anhedonia,None


In [13]:
# Fix dataframe to remove these features
remove = (tempfilt==False) \
    & scaledf['Varname'].isna()==False

scaledf[remove]

,Heading,Scale,Varname
2,DEPRESSION,Depressed Affect,A1PDEPAF
3,DEPRESSION,Depressed Affect,A1PDEPAD
5,DEPRESSION,Anhedonia,A1PANHED
6,DEPRESSION,Anhedonia,A1PANHDX
8,DEPRESSION,Depression,A1PDEPRE
9,DEPRESSION,Depression,A1PDEPDX
12,GENERALIZED ANXIETY DISORDER,Anxiety Disorder,A1PANXIE
13,GENERALIZED ANXIETY DISORDER,Anxiety Disorder,A1PANXTD
16,PANIC ATTACK (PANIC DISORDER),Scales/Items,A1PPANIC
17,PANIC ATTACK (PANIC DISORDER),Scales/Items,A1PPANDX


In [14]:
scaledf[195:]

,Heading,Scale,Varname
195,RELIGIOSITY,None,None
196,PERCEIVED DISCRIMINATION,None,None
197,PERCEIVED DISCRIMINATION,None,None
198,PERCEIVED DISCRIMINATION,None,None
199,CHRONIC CONDITIONS,None,None
200,CHRONIC CONDITIONS,Having Chronic Condition,None
201,CHRONIC CONDITIONS,Having Chronic Condition,A1SCHROX
202,PRESCRIPTION MEDICINE,None,None
203,PRESCRIPTION MEDICINE,Number of medicine taking (in past 30 days),None
204,PRESCRIPTION MEDICINE,Number of medicine taking (in past 30 days),A1SRXMED


In [15]:
vartext[]

SyntaxError: invalid syntax (<ipython-input-15-d137fd0b16b6>, line 1)

In [ ]:
vartext

In [ ]:
scaledf['Scale']

In [ ]:
scaledf.head()